In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.efficientnet import preprocess_input

/Users/hmelihalan/PycharmProjects/JupyterProject3/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('/Users/hmelihalan/PycharmProjects/JupyterProject3/2025-bamboo-summer-competiton-dl-pr/train.csv')
df['filename'] = df['filename'].astype(str)

# Split with stratification
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# UPDATED Data Augmentation with proper EfficientNet preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # REPLACED simple rescaling
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1]  # Added moderate brightness variation
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input  # SAME preprocessing for validation
)

img_size = (224, 224)
batch_size = 32

In [3]:
train_gen = train_datagen.flow_from_dataframe(
    train_df,
    directory='/Users/hmelihalan/PycharmProjects/JupyterProject3/2025-bamboo-summer-competiton-dl-pr/train',
    x_col='filename',
    y_col='label',
    target_size=img_size,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    directory='/Users/hmelihalan/PycharmProjects/JupyterProject3/2025-bamboo-summer-competiton-dl-pr/train',
    x_col='filename',
    y_col='label',
    target_size=img_size,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False  # Important for validation
)

Found 4159 validated image filenames belonging to 75 classes.
Found 1040 validated image filenames belonging to 75 classes.


In [4]:
num_classes = len(train_gen.class_indices)

# Load base model
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# NEW: Partial unfreezing (unfreeze top 100/237 layers)
base_model.trainable = True
for layer in base_model.layers[:137]:  # Freeze first 137 layers
    layer.trainable = False

# UPDATED Classifier head with BatchNorm and more dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)  # NEW
x = Dropout(0.5)(x)  # Increased from 0.3
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)  # NEW additional dropout
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

2025-07-03 16:31:47.338252: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-07-03 16:31:47.338292: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-07-03 16:31:47.338298: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-07-03 16:31:47.338320: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-03 16:31:47.338333: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
model.compile(
    optimizer=Adam(learning_rate=1e-3, clipvalue=0.5),  # Higher LR + clipping
    loss=CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

# Callbacks remain the same
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)

# First training phase
print("Initial training with frozen base...")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=25,
    callbacks=[early_stop, lr_reduce],
    verbose=1
)

# NEW: Gradual unfreezing for fine-tuning
print("\nUnfreezing more layers...")
for layer in base_model.layers[100:]:
    layer.trainable = True

# Recompile with lower LR
model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Reduced from 1e-5
    loss=CategoricalCrossentropy(label_smoothing=0.05),
    metrics=['accuracy']
)

# Fine-tuning phase
print("Fine-tuning all unfrozen layers...")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,  # Increased from 5
    callbacks=[early_stop, lr_reduce],
    verbose=1
)

Initial training with frozen base...


/Users/hmelihalan/PycharmProjects/JupyterProject3/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25


2025-07-03 16:31:53.237481: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


130/130 ━━━━━━━━━━━━━━━━━━━━ 78s 509ms/step - accuracy: 0.2639 - loss: 4.6532 - val_accuracy: 0.7808 - val_loss: 1.6046 - learning_rate: 0.0010
Epoch 2/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 108s 834ms/step - accuracy: 0.6773 - loss: 2.2587 - val_accuracy: 0.8212 - val_loss: 1.7275 - learning_rate: 0.0010
Epoch 3/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 103s 786ms/step - accuracy: 0.7551 - loss: 1.9995 - val_accuracy: 0.8462 - val_loss: 1.8058 - learning_rate: 0.0010
Epoch 4/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 92s 702ms/step - accuracy: 0.7830 - loss: 1.8520 - val_accuracy: 0.8913 - val_loss: 1.5479 - learning_rate: 0.0010
Epoch 5/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 97s 739ms/step - accuracy: 0.8262 - loss: 1.6860 - val_accuracy: 0.8875 - val_loss: 1.5871 - learning_rate: 0.0010
Epoch 6/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 113s 860ms/step - accuracy: 0.8487 - loss: 1.5653 - val_accuracy: 0.8933 - val_loss: 1.5671 - learning_rate: 0.0010
Epoch 7/25
130/130 ━━━━━━━━━━━━━━━━━━━━ 102s 782ms/step - accuracy: 0.8725 - l

In [8]:
test_csv = pd.read_csv('/Users/hmelihalan/PycharmProjects/JupyterProject3/2025-bamboo-summer-competiton-dl-pr/test.csv')

test_gen = val_datagen.flow_from_dataframe(
    test_csv,
    directory='/Users/hmelihalan/PycharmProjects/JupyterProject3/2025-bamboo-summer-competiton-dl-pr/test',
    x_col='filename',
    y_col=None,
    target_size=img_size,
    class_mode=None,
    batch_size=batch_size,
    shuffle=False
)

preds = model.predict(test_gen)
predicted_indices = np.argmax(preds, axis=1)

label_map = {v: k for k, v in train_gen.class_indices.items()}
predicted_labels = [label_map[i] for i in predicted_indices]

submission_df = test_csv.copy()
submission_df['label'] = predicted_labels
submission_df.to_csv('submission.csv', index=False)

Found 1300 validated image filenames.


/Users/hmelihalan/PycharmProjects/JupyterProject3/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


41/41 ━━━━━━━━━━━━━━━━━━━━ 12s 211ms/step
